In [1]:
from configuration import Config
from utils import CharaterTable, Preprocessor, vetorize_caption
from CaptionModel import CaptionModel

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
config = Config()
data = Preprocessor(config)
ctable = CharaterTable(data.train_captions + data.val_captions)
Y_train = vetorize_caption(data.train_captions, ctable, data.caption_len)
Y_val = vetorize_caption(data.val_captions, ctable, data.caption_len)

In [4]:
np.sum(Y_val[0][1], axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [5]:
ctable.indices_char

{0: u'',
 1: u',',
 2: u'-',
 3: u'.',
 4: u'0',
 5: u'1',
 6: u'2',
 7: u'3',
 8: u'4',
 9: u'5',
 10: u'6',
 11: u'7',
 12: u'8',
 13: u'9',
 14: u':',
 15: u'A',
 16: u'B',
 17: u'C',
 18: u'D',
 19: u'E',
 20: u'F',
 21: u'G',
 22: u'H',
 23: u'I',
 24: u'J',
 25: u'K',
 26: u'L',
 27: u'M',
 28: u'N',
 29: u'O',
 30: u'P',
 31: u'R',
 32: u'S',
 33: u'T',
 34: u'U',
 35: u'V',
 36: u'W',
 37: u'X',
 38: u'Y',
 39: u'a',
 40: u'b',
 41: u'c',
 42: u'd',
 43: u'e',
 44: u'f',
 45: u'g',
 46: u'h',
 47: u'i',
 48: u'j',
 49: u'k',
 50: u'l',
 51: u'm',
 52: u'n',
 53: u'o',
 54: u'p',
 55: u'r',
 56: u's',
 57: u't',
 58: u'u',
 59: u'v',
 60: u'w',
 61: u'x',
 62: u'y',
 63: u'z',
 64: u'\xb7',
 65: u'\u200b',
 66: u'\u201c',
 67: u'\u201d',
 68: u'\u3001',
 69: u'\u4e00',
 70: u'\u4e01',
 71: u'\u4e03',
 72: u'\u4e07',
 73: u'\u4e08',
 74: u'\u4e09',
 75: u'\u4e0a',
 76: u'\u4e0b',
 77: u'\u4e0d',
 78: u'\u4e0e',
 79: u'\u4e10',
 80: u'\u4e11',
 81: u'\u4e13',
 82: u'\u4e14',
 83: 